In [22]:
from generalConversion import createDataFromPath

import os
import sys
from generalConversion import createDataFromPath


#XGBoost
import xgboost as xgb

# Save the original standard output
original_stdout = sys.stdout

# Redirect standard output to a null device
sys.stdout = open(os.devnull, 'w')

# Call the function
data, salePrice = createDataFromPath("train.csv", True)
data_create = createDataFromPath("test.csv", False)

# Restore the original standard output
sys.stdout = original_stdout

data.shape


(1460, 105)

In [23]:
data_train = data[:int(data.shape[0]*0.8)]
data_test = data[int(data.shape[0]*0.8):]
salePrice_train = salePrice[:int(data.shape[0]*0.8)]
salePrice_test = salePrice[int(data.shape[0]*0.8):]

#XGBoost
dtrain = xgb.DMatrix(data_train, label=salePrice_train)
dtest = xgb.DMatrix(data_test, label=salePrice_test)
dcreate = xgb.DMatrix(data_create)

# Define the list of max_depth values to try
max_depths = np.arange(1, 10)

# Initialize best_rmse and best_max_depth
best_rmse = float("Inf")
best_max_depth = None

# Loop over max_depths
for max_depth in max_depths:
    # Update the max_depth value in the parameter grid
    param_grid = {'max_depth': max_depth, 'eta': 1, 'objective': 'reg:squarederror'}

    # Perform cross-validation
    cv_results = xgb.cv(param_grid, dtrain, num_boost_round=2, nfold=5, metrics='rmse', early_stopping_rounds=10)

    # Get the mean RMSE of the last boosting round
    mean_rmse = cv_results['test-rmse-mean'].iloc[-1]

    # Update best_rmse and best_max_depth if this RMSE is lower
    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_max_depth = max_depth

# Print the best max_depth
print('Best max_depth: ', best_max_depth)




Best max_depth:  9


In [25]:
# Define the list of num_round values to try
num_rounds = np.arange(1, 10)

# Initialize best_rmse and best_num_round
best_rmse = float("Inf")
best_num_round = None

# Set the max_depth to the best value found previously
param_grid = {'max_depth': best_max_depth, 'eta': 1, 'objective': 'reg:squarederror'}

# Loop over num_rounds
for num_round in num_rounds:
    # Perform cross-validation
    cv_results = xgb.cv(param_grid, dtrain, num_boost_round=num_round, nfold=5, metrics='rmse', early_stopping_rounds=10)

    # Get the mean RMSE of the last boosting round
    mean_rmse = cv_results['test-rmse-mean'].iloc[-1]

    # Update best_rmse and best_num_round if this RMSE is lower
    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_num_round = num_round

# Print the best num_round
print('Best num_round: ', best_num_round)


Best num_round:  2


In [29]:

param = {'max_depth': best_max_depth, 'eta': 1, 'objective': 'reg:squarederror'}
num_round = best_num_round
bst = xgb.train(param, dtrain, num_round)
# make prediction
preds = bst.predict(dtest)

print(salePrice_test[0], preds[0])

#calculate error with RMSE
from sklearn.metrics import mean_squared_error
from math import sqrt
import numpy as np

rms = sqrt(mean_squared_error(np.log(salePrice_test), np.log(preds)))
print(rms)

235000.0 192877.42
0.1796114817867331


In [27]:
#Create submission file
preds_create = bst.predict(dcreate)
import csv
with open('submission.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(['Id', 'SalePrice'])
    for i in range(len(preds_create)):
        writer.writerow([i+1461, preds_create[i]])

